1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [20]:
# If haven't downloaded it yet, please get the data file with wget
# !wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

import pandas as pd
import numpy as np

# I use Windows, so no wget for me. The following lines should work fine
# on both Windows and Linux machines. However, it does not create the directory data
# if it does not exist.
import urllib.request
# moreover, the original link in the command above was downloading the html
# file of the Dropbox preview page and not the actual txt file. I had to append
# '?dl=1' to the link to have direct download access to the file
# (source for this: Dropbox documentation)
url = 'https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt?dl=1'
filename = 'data/data_000637.txt'
urllib.request.urlretrieve(url, filename)

('data/data_000637.txt', <http.client.HTTPMessage at 0x26c3f5304d0>)

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [30]:
N = 50
dataSet = pd.read_csv(filename, sep=",", header=0, nrows=N)
dataSet

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [38]:
out = dataSet.groupby('ORBIT_CNT').count()
print(out)

            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS
ORBIT_CNT                                                
3869200167    43    43           43          43        43
3869200168     7     7            7           7         7


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.